In [1]:
from libfunc_trip_order import *

In [2]:
'''
Расчет оптимального пробега на каждый день недели для точек
На вход: необходимо ввести путь к файлу
Расписание ДОЛЖНО содержать Новый для КПК первой точкой на день с минимальным индесом (0 или 1)
Вывод: таблица расстояний и запись распиания в файл excel
'''
schedule_data = open_excel_file(
    file_path='C:\\Users\\andre\\OneDrive\\Рабочий стол\\sav\\iorder\\DP_iorder_Боженко.xlsx')
# Словарь с точками на четную и не четную неделю
full_dict = schedule_odd_even(schedule_data)

data = calc_trips_order(full_dict)
# Отображение таблицы расстояний
display(df2table(data[0]))
# Запись  файл excel
write_to_excel(data[1])
print('Файл создан')

,1-ПН,2-ВТ,3-СР,4-ЧТ,5-ПТ
четная нед.,103.8868,0.0000,17.4726,88.2912,257.6436
не четная нед.,117.7402,276.2076,0.0000,82.4906,277.1025


Файл создан


In [2]:
'''
Расчет оптимальных пробегов для всех расписаний в папке
На вход: путь к папке с файлами расписаний
Вывод: таблицы с пробегами по каждому дню
'''
# Путь к папке с расписаниями
path = 'C:\\Users\\andre\\OneDrive\\Рабочий стол\\sav\\DP\\Тест\\12'

# Вызов функции
calc_order_for_folder(path) 

IndexError: list index out of range